In [1]:
import os 
import warnings
import time
import numpy as np
import scipy.stats
import sys
import sklearn
import sklearn.datasets

from pyspark.sql import SparkSession
warnings.filterwarnings('ignore')
import pandas as pd

# launch this cell if you have issues on windows with py4j (think about updating your PATH)

os.environ['PYSPARK_DRIVER_PYTHON_OPTS']= "notebook"
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['PYSPARK_PYTHON'] = sys.executable

# starts a spark session from notebook

os.environ['PYSPARK_SUBMIT_ARGS'] ="--conf spark.driver.memory=4g  pyspark-shell"


spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("feature_selection") \
    .getOrCreate()

sc=spark.sparkContext

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/25 08:47:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/25 08:47:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/05/25 08:47:17 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/05/25 08:47:17 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/05/25 08:47:17 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [2]:
train_sessions_engineered = spark.read.csv('../Data/session_engineered_features.txt',header=False,
                                          inferSchema=True)

train_purchases = spark.read.load('../Data/train_purchases.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')

In [3]:
# sort by session
X = train_sessions_engineered.orderBy('_c0') 
X = X.drop('_c0')
X.take(10)

X_np = np.array(X.collect())
X_np_split = X_np[:10000]
print(X_np_split.shape)
t_X = np.transpose(X_np_split)
n_total_features = t_X.shape[0]
print(t_X)
# we want the nb partition to be between 2 or 3 times more than the number of core in our computer
Nb_partition=10
X_RDD = sc.parallelize(t_X,Nb_partition)


# Counting the number of rows will allow to implicitly cache the data
#print(X_RDD.count())
#X_RDD.take(10)

22/05/25 08:47:26 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


(10000, 38)
[[312.   0. 163. ...   0.   0.   0.]
 [  3.   0.   2. ...   1.   3.   0.]
 [  5.   4.   4. ...   3.   1.   2.]
 ...
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]]


In [4]:
train_purchases_reformated = train_purchases.drop('date').withColumnRenamed('item_id', 'item_id_purchased')

Y_order_by_session_id = train_purchases_reformated.drop('session_id')

#Y_order_by_session_id.take(10)
Y_numpy = np.array(Y_order_by_session_id.collect())
Y_numpy = Y_numpy.flatten()
print(Y_numpy)
Y_numpy_split = Y_numpy[:10000]

[15085 18626 24911 ... 21630 16962 16631]


In [5]:
print(X.count())
len(X_np_split) == len(Y_numpy_split)

1000000


True

# Feature Selection

After the feature engineering, we could have a lot of features which are not really worth to give to the predicting model. Considering this problem, we want to use some feature selection algorithms to take the feature which are the most interesting to the model.

We were asked to implement two scalable feature selection algorithms, a ranking algorithm and a forward feature selection.

We could be interested in Minimum-redundancy-maximum-relevance (mRMR) feature selection.


In [6]:
'''
def generate_dataset(n_samples=100, n_informative=1, n_noisy=2, n_redundant=1, random_seed=0):
    """
    generate a dataset to test
    """
    # Set random seed
    np.random.seed(random_seed)
    
    # Use sklearn.datasets.make_regression to generate an artificial dataset where the output y 
    # is correlated with a subset of of the input features
    X, Y = sklearn.datasets.make_classification(n_samples=n_samples, 
                                            n_features=n_informative+n_noisy, 
                                            n_informative=n_informative)
    
    # Create a random mixing matrix for generating redundant features from informative ones
    mixing_matrix = np.random.random((n_informative, n_redundant))
    
    # Create redundant features by taking random linear combinations of informative features
    redundant_features = np.dot(X[:,0:n_informative], mixing_matrix)
    
    # Add redundant features to the input data
    X = np.concatenate((X, redundant_features), axis=1)
    
    # Return input data X, output data Y
    return X, Y


# Let us generate the dataset
N = 1000
n_informative = 100
n_noisy = 100
n_redundant = 100

X, Y = generate_dataset(n_samples=N, 
                        n_informative=n_informative, 
                        n_noisy=n_noisy, 
                        n_redundant=n_redundant)

t_X = np.transpose(X)

B=3
X_RDD=sc.parallelize(t_X,B).cache()

# Counting the number of rows will allow to implicitly cache the data
X_RDD.count()
X_RDD.take(10)

print(type(Y))'''

'\ndef generate_dataset(n_samples=100, n_informative=1, n_noisy=2, n_redundant=1, random_seed=0):\n    """\n    generate a dataset to test\n    """\n    # Set random seed\n    np.random.seed(random_seed)\n    \n    # Use sklearn.datasets.make_regression to generate an artificial dataset where the output y \n    # is correlated with a subset of of the input features\n    X, Y = sklearn.datasets.make_classification(n_samples=n_samples, \n                                            n_features=n_informative+n_noisy, \n                                            n_informative=n_informative)\n    \n    # Create a random mixing matrix for generating redundant features from informative ones\n    mixing_matrix = np.random.random((n_informative, n_redundant))\n    \n    # Create redundant features by taking random linear combinations of informative features\n    redundant_features = np.dot(X[:,0:n_informative], mixing_matrix)\n    \n    # Add redundant features to the input data\n    X = np.conc

In [7]:
def get_score_mrmr(x, y):
    """
    return the correlation value between two variable in absolute value
    """

    return np.abs(scipy.stats.pearsonr(x, y)[0])

In [8]:
def get_mrmr_score_spark(x,selected_features):
    """
    x : the feature to evaluate to add into the model
    y : output value
    selected_features : the features already selected
    return the score for x with the rest of selected variables
    """
    
    y = broadcast_Y.value
    # Get correlation score between feature x and output y (relevance)
    score_x_y_s = get_score_mrmr(x, y)
    
    
    nb_selected_features = selected_features.shape[0]
    # If some features have already been selected
    if nb_selected_features > 0:
        
        # Get corrrelation scores between x and each feature already selected (redundancy)
        score_features_x_s = np.zeros(nb_selected_features, dtype=float)
        
        for j in range(nb_selected_features):
            
            score_x_s_j = get_score_mrmr(x, selected_features[j,:])
            
            # if score is nan considering that we want to calculate the mean
            # we transform it in 0 ?
            if np.isnan(score_x_s_j):
                score_x_s_j=0
            
                
            score_features_x_s[j] = score_x_s_j
        
        print(score_x_s_j)
                
        # Final score is relevance to output feature - average redundancy with already selected features
        score_x_y_s = score_x_y_s - np.mean(score_features_x_s)
        
    return score_x_y_s

In [9]:
def mrmr_spark(n_total_features, K, sc, X_RDD):
    """
    n_total_features : number of total features
    K : number of feature to select
    sc : spark context
    X_RDD : RDD of the variable X
    Y: Output data
    
    return the indice of selected features and time execution using mrmr
    """
    time_execution = []
    remaining_features_indices = list(range(n_total_features))
    selected_features_indices = []
    

    for k in range(K):
        print("Step: "+str(k))
    
        start_time=time.time()
        # Get the subset of selected features values, and cast as an array
        selected_features = X_RDD.zipWithIndex().filter(lambda x: x[1] in selected_features_indices).map(lambda x: x[0]).collect()
        selected_features = np.array(selected_features)
        print(selected_features)
    
        # mRMR scores are computed by first filtering `t_X` to remove already selected features, and then mapping 
        # each remaining feature using the `get_mrmr_score_spark` function
        scores = X_RDD.zipWithIndex().filter(lambda x: x[1] in remaining_features_indices).map(lambda x:get_mrmr_score_spark(x[0],selected_features)).collect()
    
        # Once all mRMR scores are computed, the index of the feature with the highest score is selected
        scores = np.array(scores)
        
    
        index_max_score_features = np.argmax(scores)
    
        selected_features_indices.append(remaining_features_indices[index_max_score_features])
    
        del(remaining_features_indices[index_max_score_features])
    
        print(time.time()-start_time)
        time_execution.append(time.time()-start_time)
        
    return selected_features_indices, time_execution

In [10]:
broadcast_Y = sc.broadcast(Y_numpy_split)
selected_features_indices, execution_time = mrmr_spark(n_total_features, 20, sc,X_RDD)

Step: 0
[]


/home/guest/anaconda3/lib/python3.9/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


1.1576128005981445
Step: 1
[[0. 0. 0. ... 0. 0. 0.]]


0
0
/home/guest/anaconda3/lib/python3.9/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/guest/anaconda3/lib/python3.9/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/guest/anaconda3/lib/python3.9/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


0.6671199798583984
Step: 2
[[2020. 2020. 2020. ... 2020. 2020. 2021.]
 [   0.    0.    0. ...    0.    0.    0.]]


...

0.5731711387634277
Step: 18
[[  312.     0.   163. ...     0.     0.     0.]
 [ 2020.  2020.  2020. ...  2020.  2020.  2021.]
 [ 9655. 15654. 18316. ... 23456. 15624. 27244.]
 ...
 [    0.     0.     0. ...     0.     0.     0.]
 [    0.     0.     0. ...     0.     0.     0.]
 [    0.     0.     0. ...     0.     0.     0.]]


00

0
0
0
0
0
00

0
00

0
0
0
00

0
0
0


0.5450198650360107
Step: 19
[[  312.     0.   163. ...     0.     0.     0.]
 [ 2020.  2020.  2020. ...  2020.  2020.  2021.]
 [ 9655. 15654. 18316. ... 23456. 15624. 27244.]
 ...
 [    0.     0.     0. ...     0.     0.     0.]
 [    0.     0.     0. ...     0.     0.     0.]
 [    0.     0.     0. ...     0.     0.     0.]]


0
0
0
0
00

00

0
0
0
0
0
0
0
0
0
0
0


0.6271693706512451


In [11]:
selected_features_indices
#selected_features = X_RDD.zipWithIndex().filter(lambda x: x[1] in selected_features_indices).map(lambda x: x[0]).collect()

#print(selected_features)

[36, 4, 0, 34, 17, 35, 18, 32, 5, 33, 16, 31, 21, 19, 28, 26, 30, 14, 23, 13]

Then, we will look a the forward feature Selection by training on a decision tree with every feature and add the features which get the best results.

In [18]:
from sklearn import tree
from sklearn import linear_model
from sklearn.model_selection import train_test_split

def fit_get_score(x,selected_features):
    """
    train an model to get the score with the addition of a specified x feature
    """    
    
    y = broadcast_Y.value
    n_selected_features = selected_features.shape[0]
    
    if n_selected_features>0:
        # need to merge x and the already selected features
        x = np.vstack((selected_features,x))
        x = x.transpose()
    else:
        x = x.reshape(-1, 1)
        
    
    #split data
    x_train,x_test,y_train,y_test = train_test_split(x,y, test_size= 0.3)
    
    
    
    # train on training data
    model = tree.DecisionTreeClassifier()
    model = model.fit(x_train,y_train)
    
    print(x_test.shape)
    
    # get score on testing set
    return model.score(x_test, y_test)

In [19]:
def forward_feature_selection(n_total_features, K, sc, X_RDD):
    """
    n_total_features : number of total features
    K : number of feature to select
    sc : spark context
    X_RDD : RDD of the variable X
    Y: Output data
    
    return the indice of selected features and time execution
    by using a decision tree as model to calculate the score
    """
    time_execution = []
    
    remaining_features_indices = list(range(n_total_features))
    selected_features_indices = []
    
    for k in range(K):
        print("Step: "+str(k))
    
        start_time=time.time()

        # Get the subset of selected features values, and cast as an array
        selected_features = X_RDD.zipWithIndex().filter(lambda x: x[1] in selected_features_indices).map(lambda x: x[0]).collect()
        selected_features = np.array(selected_features)
        
    
        #  scores for a certain model are computed by first filtering `t_X` to remove already selected features, and then mapping 
        # each remaining feature using the `fit_get_score` function
        scores = X_RDD.zipWithIndex().filter(lambda x: x[1] in remaining_features_indices).map(lambda x:fit_get_score(x[0],selected_features)).collect()
    
        # Once all scores are computed, the index of the feature with the highest value is chosen
        scores = np.array(scores)
        
        print("best_score :", np.max(scores))
    
        index_max_score_features = np.argmax(scores)
    
        selected_features_indices.append(remaining_features_indices[index_max_score_features])
    
        del(remaining_features_indices[index_max_score_features])
    
        print(time.time()-start_time)
        time_execution.append(time.time()-start_time)
        
    return selected_features_indices, time_execution
    

In [20]:
selected_features_indices_forward, execution_time_forward = forward_feature_selection(n_total_features, 5, sc,X_RDD)
selected_features_indices_forward

Step: 0


(3000, 1):>                                                        (0 + 4) / 10]
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1):===========>                                             (2 + 4) / 10]
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1):======================>                                  (4 + 4) / 10]
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1):============================>                            (5 + 4) / 10]
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1):=============================================>           (8 + 2) / 10]
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1):===================================================>     (9 + 1) / 10]


best_score : 0.019
3.212231159210205
Step: 1


(3000, 2):>                                                        (0 + 4) / 10]
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2):=====>                                                   (1 + 4) / 10]
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2):===========>                                             (2 + 4) / 10]
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2):=================>                                       (3 + 4) / 10]
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2):============================>                            (5 + 4) / 10]
(3000, 2):==================================>                      (6 + 4) / 10]
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2):=======================================>                 (7 + 3) / 10]
(3000, 2):=============================================>           (8 + 2) / 10]
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2):===================================================>     (9 + 

best_score : 0.021
14.44019103050232
Step: 2


...

best_score : 0.021
15.035150289535522
Step: 4


(3000, 5)0:>                                                       (0 + 4) / 10]
(3000, 5)
(3000, 5)
(3000, 5)
(3000, 5)
(3000, 5)0:=====>                                                  (1 + 4) / 10]
(3000, 5)
(3000, 5)
(3000, 5)
(3000, 5)
(3000, 5)
(3000, 5)0:===========>                                            (2 + 4) / 10]
(3000, 5)
(3000, 5)
(3000, 5)
(3000, 5)
(3000, 5)0:================>                                       (3 + 4) / 10]
(3000, 5)
(3000, 5)0:======================>                                 (4 + 4) / 10]
(3000, 5)
(3000, 5)0:============================>                           (5 + 4) / 10]
(3000, 5)
(3000, 5)0:=================================>                      (6 + 4) / 10]
(3000, 5)
(3000, 5)
(3000, 5)
(3000, 5)0:=======================================>                (7 + 3) / 10]
(3000, 5)
(3000, 5)
(3000, 5)
(3000, 5)0:============================================>           (8 + 2) / 10]
(3000, 5)
(3000, 5)


best_score : 0.020666666666666667
14.24113655090332


(3000, 5)


[5, 28, 22, 36, 14]

In [15]:
def forward_feature_selection_diff(n_total_features, inc_accuracy, sc, X_RDD):
    """
    n_total_features : number of total features
    inc_accuracy : increase of accuracy needed to continue the algorithm
    sc : spark context
    X_RDD : RDD of the variable X
    Y: Output data
    
    return the indice of selected features and time execution
    by using a decision tree as model to calculate the score
    """
    time_execution = []
    
    remaining_features_indices = list(range(n_total_features))
    selected_features_indices = []
    
    last_best_score = 0
    diff_accuracy = 1
    k = 0
    while diff_accuracy > inc_accuracy:
        print("Step: "+str(k))
    
        start_time=time.time()

        # Get the subset of selected features values, and cast as an array
        selected_features = X_RDD.zipWithIndex().filter(lambda x: x[1] in selected_features_indices).map(lambda x: x[0]).collect()
        selected_features = np.array(selected_features)
        
    
        #  scores for a certain model are computed by first filtering `t_X` to remove already selected features, and then mapping 
        # each remaining feature using the `fit_get_score` function
        scores = X_RDD.zipWithIndex().filter(lambda x: x[1] in remaining_features_indices).map(lambda x:fit_get_score(x[0],selected_features)).collect()
    
        # Once all scores are computed, the index of the feature with the highest value is chosen
        scores = np.array(scores)
        
        # compute the difference between last result and new result
        best_score = np.max(scores)
        diff_accuracy = best_score - last_best_score
        
        
        print("best_accuracy:", best_score)
        
        if best_score > last_best_score:
        
            index_max_score_features = np.argmax(scores)
    
            selected_features_indices.append(remaining_features_indices[index_max_score_features])
    
            del(remaining_features_indices[index_max_score_features])
        
        last_best_score = best_score
        print(time.time()-start_time)
        time_execution.append(time.time()-start_time)
        
        k += 1
        
    return selected_features_indices, time_execution

In [21]:
selected_features_indices_forward_diff, execution_time_forward_diff = forward_feature_selection_diff(n_total_features, 0.0001, sc,X_RDD)
selected_features_indices_forward_diff

Step: 0


(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)4:>                                                       (0 + 4) / 10]
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)4:===========>                                            (2 + 4) / 10]
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)4:======================>                                 (4 + 4) / 10]
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)4:============================>                           (5 + 4) / 10]
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)4:============================================>           (8 + 2) / 10]
(3000, 1)
(3000, 1)4:==================================================>     (9 + 1) / 10]


best_accuracy: 0.015666666666666666
2.7608773708343506
Step: 1


(3000, 2)8:>                                                       (0 + 4) / 10]
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)8:=====>                                                  (1 + 4) / 10]
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)8:===========>                                            (2 + 4) / 10]
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)8:================>                                       (3 + 4) / 10]
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)8:======================>                                 (4 + 4) / 10]
(3000, 2)8:============================>                           (5 + 4) / 10]
(3000, 2)
(3000, 2)
(3000, 2)8:=================================>                      (6 + 4) / 10]
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)8:=======================================>                (7 + 3) / 10]
(3000, 2)
(3000, 2)8:============================================>           (8 + 2) / 10]
(3000, 2)
(3000, 2)
(3000, 2)
(

best_accuracy: 0.021
14.777044534683228
Step: 2


(3000, 3)2:>                                                       (0 + 4) / 10]
(3000, 3)
(3000, 3)
(3000, 3)
(3000, 3)
(3000, 3)2:=====>                                                  (1 + 4) / 10]
(3000, 3)
(3000, 3)
(3000, 3)
(3000, 3)
(3000, 3)
(3000, 3)2:===========>                                            (2 + 4) / 10]
(3000, 3)
(3000, 3)
(3000, 3)
(3000, 3)
(3000, 3)2:================>                                       (3 + 4) / 10]
(3000, 3)2:======================>                                 (4 + 4) / 10]
(3000, 3)
(3000, 3)
(3000, 3)2:============================>                           (5 + 4) / 10]
(3000, 3)
(3000, 3)2:=================================>                      (6 + 4) / 10]
(3000, 3)
(3000, 3)
(3000, 3)
(3000, 3)
(3000, 3)
(3000, 3)
(3000, 3)2:=======================================>                (7 + 3) / 10]
(3000, 3)
(3000, 3)
(3000, 3)2:============================================>           (8 + 2) / 10]
(3000, 3)
(3000, 3)2:==========

best_accuracy: 0.020666666666666667
14.918395042419434


(3000, 3)


[5, 25]

In [17]:
#spark.stop()